In [32]:
import os
import glob
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from scipy.spatial import distance
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None

import colormath
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

%matplotlib inline

In [2]:
def rgb2lab ( inputColor ) :

   num = 0
   RGB = [0, 0, 0]

   for value in inputColor :
       value = float(value) / 255

       if value > 0.04045 :
           value = ( ( value + 0.055 ) / 1.055 ) ** 2.4
       else :
           value = value / 12.92

       RGB[num] = value * 100
       num = num + 1

   XYZ = [0, 0, 0,]

   X = RGB [0] * 0.4124 + RGB [1] * 0.3576 + RGB [2] * 0.1805
   Y = RGB [0] * 0.2126 + RGB [1] * 0.7152 + RGB [2] * 0.0722
   Z = RGB [0] * 0.0193 + RGB [1] * 0.1192 + RGB [2] * 0.9505
   XYZ[ 0 ] = round( X, 4 )
   XYZ[ 1 ] = round( Y, 4 )
   XYZ[ 2 ] = round( Z, 4 )

   XYZ[ 0 ] = float( XYZ[ 0 ] ) / 95.047         # ref_X =  95.047   Observer= 2°, Illuminant= D65
   XYZ[ 1 ] = float( XYZ[ 1 ] ) / 100.0          # ref_Y = 100.000
   XYZ[ 2 ] = float( XYZ[ 2 ] ) / 108.883        # ref_Z = 108.883

   num = 0
   for value in XYZ :

       if value > 0.008856 :
           value = value ** ( 0.3333333333333333 )
       else :
           value = ( 7.787 * value ) + ( 16 / 116 )

       XYZ[num] = value
       num = num + 1

   Lab = [0, 0, 0]

   L = ( 116 * XYZ[ 1 ] ) - 16
   a = 500 * ( XYZ[ 0 ] - XYZ[ 1 ] )
   b = 200 * ( XYZ[ 1 ] - XYZ[ 2 ] )

   Lab [ 0 ] = round( L, 4 )
   Lab [ 1 ] = round( a, 4 )
   Lab [ 2 ] = round( b, 4 )

   return Lab

In [3]:
def ColorDistance(rgb1,rgb2):
    color1_rgb = sRGBColor(rgb1[0], rgb1[1], rgb1[2])

    # Blue Color
    color2_rgb = sRGBColor(rgb2[0], rgb2[1], rgb2[2])

    # Convert from RGB to Lab Color Space
    color1_lab = convert_color(color1_rgb, LabColor)

    # Convert from RGB to Lab Color Space
    color2_lab = convert_color(color2_rgb, LabColor)

    # Find the color difference
    delta_e = delta_e_cie2000(color1_lab, color2_lab)
    
    return delta_e

In [4]:
def resize_image(image_to_resize, scale_factor):
    image_aspect_ratio = image_to_resize.size[0] / image_to_resize.size[1]
    
    new_x = int(np.rint(image_to_resize.size[0] / scale_factor))
    new_y = int(np.rint(image_to_resize.size[1] / scale_factor))
    
    resized_image = image_to_resize.resize((new_x, new_y), Image.ANTIALIAS)
    return resized_image

In [5]:
def filter_colors_by_percent(image, percent_threshold):
    total_pixels = image.size[0]*image.size[1]
    print('total_pixels = ', total_pixels)
    colors = image.getcolors(maxcolors=total_pixels)
    
    colors_above_threshold_percent = []
    
    for color in colors:
        if color[0]/total_pixels < percent_threshold: 
            pass
        else:
            colors_above_threshold_percent.append(color)
            
    return colors_above_threshold_percent

In [8]:
def match_color_to_rainbow(color, threshold):
    colors = ['red', 'red', 'red', 'red',
              'red', 'red', 'red', 'red',
             'red', 'red']
    
#     red = np.array([128, 0, 0])
#     orange = np.array([128, 64, 0])
#     yellow = np.array([128, 128, 0])
#     green = np.array([0, 128, 0])
#     blue = np.array([0, 0, 128])
#     indigo = np.array([46, 43, 95])
    red_1 = np.array([102, 0, 0])
    red_2 = np.array([128, 0, 0])
    red_3 = np.array([153, 0, 0])
    red_4 = np.array([179, 0, 0])
    red_5 = np.array([204, 0, 0])
    red_6 = np.array([230, 0, 0])
    red_7 = np.array([255, 0, 0])
    red_8 = np.array([255, 26, 26])
    red_9 = np.array([255, 51, 51])
    red_10 = np.array([255, 77, 77])
    
    all_distances = np.array([ColorDistance(color, red_1),
                              ColorDistance(color, red_2),
                    ColorDistance(color, red_3), ColorDistance(color, red_4),
                    ColorDistance(color, red_5), ColorDistance(color, red_6),
                    ColorDistance(color, red_7), ColorDistance(color, red_8),
                             ColorDistance(color, red_9), ColorDistance(color, red_10)])
    
#     all_distances = np.array([distance.euclidean(color, red), distance.euclidean(color, orange),
#                     distance.euclidean(color, yellow), distance.euclidean(color, green),
#                     distance.euclidean(color, blue), distance.euclidean(color, indigo),
#                     distance.euclidean(color, violet)])
    
    #print('min distance distance = ', np.min(all_distances))
    if np.min(all_distances) > threshold:
        return 'none'
    else:
        return colors[np.argmin(all_distances)]

In [9]:
def take_image_colors_return_binary_presence(all_colors):
    color_vector = [0]
    
#     if 'red' in all_colors:
#         color_vector[0] = 1
        
#     if 'orange' in all_colors:
#         color_vector[1] = 1
        
#     if 'yellow' in all_colors:
#         color_vector[2] = 1
        
#     if 'green' in all_colors:
#         color_vector[3] = 1
        
#     if 'blue' in all_colors:
#         color_vector[4] = 1
    
#     if 'indigo' in all_colors:
#         color_vector[5] = 1
    
    if 'red' in all_colors:
        color_vector[0] = 1
        
    return color_vector

In [33]:
all_art_files = glob.glob('Artworks/Romanticism/*')

# all_art_files = []
# for file in all_art_files_raw:
#     all_art_files.append(file.replace(' ', ''))

In [31]:
df = pd.DataFrame(columns = ['image ID', 'red'])

for art_file in all_art_files: 
    img = Image.open(art_file)
    resized_image = resize_image(img, 4)
    quantized_image = resized_image.quantize(colors=256, method=None, kmeans=0, palette=None)
    quantized_image_rgb = quantized_image.convert('RGB')
    thresholded_colors = filter_colors_by_percent(quantized_image_rgb, 0.005)

    color_labels = []
    for color in thresholded_colors:
        color_labels.append(match_color_to_rainbow(color[1], 30))
    
    color_presence = take_image_colors_return_binary_presence(color_labels)
    info_for_df = color_presence
    info_for_df.insert(0, art_file)
#     print('info_for_df = ', info_for_df)
#     print()
    df.loc[len(df)] = info_for_df
    
df.to_csv('Romanticism_red_search_005_30.csv', index=False)

total_pixels =  114920
total_pixels =  1031940
total_pixels =  700290
total_pixels =  84500
total_pixels =  152490
total_pixels =  102000
total_pixels =  107310
total_pixels =  743340
total_pixels =  48048
total_pixels =  99960
total_pixels =  91016
total_pixels =  192000
total_pixels =  99960
total_pixels =  137824
total_pixels =  808000
total_pixels =  870216
total_pixels =  218268
total_pixels =  219264
total_pixels =  816000
total_pixels =  80360
total_pixels =  301344
total_pixels =  689130
total_pixels =  204120
total_pixels =  109060
total_pixels =  653125
total_pixels =  958650
total_pixels =  108015
total_pixels =  100500
total_pixels =  725402
total_pixels =  252000
total_pixels =  69657
total_pixels =  629046
total_pixels =  159903
total_pixels =  220550
total_pixels =  306688
total_pixels =  75088
total_pixels =  113460
total_pixels =  748440
total_pixels =  733194
total_pixels =  873307
total_pixels =  404460
total_pixels =  255148
total_pixels =  117740
total_pixels =  55